In [10]:
tabu_entry_t = tuple[str, int, int, int, int, int, float, int, float]


def tabu_entry(l: list[str]) -> tabu_entry_t:
    return (
        l[0][5:-4],
        int(l[1]),
        int(l[2]),
        int(l[3]),
        int(l[4]),
        int(l[5]),
        float(l[6]),
        int(l[7]),
        float(l[8])
    )


exact_entry_t = tuple[str, int, float, float, float]


def exact_entry(l: list[str]) -> exact_entry_t:
    return (
        l[0][5:-4],
        int(l[1]),
        float(l[2]),
        float(l[3]),
        float(l[4])
    )


def size_of(d: str):
    return int(d[3:-2])


def avg(l: list[int | float]):
    return sum(l)/len(l)


In [11]:
import os

raw_tabu: list[tabu_entry_t] = []
files = os.scandir('results/tabu/')
for file in files:
    with open(file.path) as f:
        lines = f.readlines()
        for l in lines:
            l = l.split('\t')
            if len(l) == 9:
                raw_tabu.append(tabu_entry(l))

raw_exact: list[tabu_entry_t] = []
files = os.scandir('results/exact/')
for file in files:
    with open(file.path) as f:
        lines = f.readlines()
        for l in lines:
            l = l.split('\t')
            if len(l) == 5:
                raw_exact.append(exact_entry(l))


In [12]:
time_map: dict[int, dict[int, list[float]]] = {}
for (_, n, _, _, nig, _, _, i, t) in raw_tabu:
    if n % 20 != 0:
        continue
    if n not in time_map:
        time_map[n] = {}
    if nig not in time_map[n]:
        time_map[n][nig] = [i]
    else:
        time_map[n][nig].append(i)

points: list[(int, list[(int, float)])] = [(n,
                                            sorted([(nig, avg(time_map[n][nig])) for nig in time_map[n]]))
                                           for n in time_map]
points.sort()

with open('report/data/nig_iter.dat', 'w') as f:
    for (n, l) in points:
        for (nig, t) in l:
            f.write(f"{n}\t{nig}\t{t}\n")
        f.write("\n")


In [13]:
# data -> tabu_size -> values
data_map: dict[str, dict[int, list[float]]] = {}
for (d, _, t, _, _, _, v, _, _) in raw_tabu:
    if d not in data_map:
        data_map[d] = {}
    if t not in data_map[d]:
        data_map[d][t] = []
    data_map[d][t].append(v)

# data -> tabu_size -> avg_value
avg_map: dict[str, dict[int, float]] = {}
for d in data_map:
    for t in data_map[d]:
        if d not in avg_map:
            avg_map[d] = {}
        avg_map[d][t] = avg(data_map[d][t])

# data -> best tabu_sizes
min_avg_map: dict[str, list[int]] = {}
for d in avg_map:
    good: list[int] = []
    m = min(avg_map[d].values())
    min_avg_map[d] = [t for t in avg_map[d] if avg_map[d][t] <= m + 1e-6]

points: list[tuple[int, int]] = []
for d in min_avg_map:
    for t in min_avg_map[d]:
        points.append((size_of(d), t))

points_w_map: dict[tuple[int, int], int] = {
    p: points.count(p) for p in points
}

points_w: list[tuple[int, int, int]] = [
    (p[0], p[1], points_w_map[p]) for p in points_w_map]
points_w.sort()

with open('report/data/tabu_size.dat', 'w') as f:
    for p in points_w:
        f.write("\t".join([str(n) for n in p])+"\n")


In [14]:
# data -> non_inc_tabu -> non_inc_global -> value
nig_values_map: dict[str, dict[int, dict[int, float]]] = {}
for (d, _, _, nit, nig, _, v, _, _) in raw_tabu:
    if d not in nig_values_map:
        nig_values_map[d] = {}
    if nit not in nig_values_map[d]:
        nig_values_map[d][nit] = {}
    if nig in nig_values_map[d][nit]:
        nig_values_map[d][nit][nig] = min(nig_values_map[d][nit][nig], v)
    else:
        nig_values_map[d][nit][nig] = v

# data -> non_inc_tabu -> list of (non_inc_globals s.t. there is improvement, improvement)
# nig_values_list_imp: dict[str, dict[int, list[tuple[int, float]]]] = {}
# for d in nig_values_map:
#     for nit in nig_values_map[d]:
#         l = []
#         nigs = sorted(nig_values_map[d][nit].keys())
#         best_v = nig_values_map[d][nit][nigs[0]]
#         for nig in nigs:
#             v = nig_values_map[d][nit][nig]
#             if v < best_v:
#                 l.append((nig, 100*(best_v-v)/best_v))
#                 best_v = v
#         if len(l) != 0:
#             if d not in nig_values_list_imp:
#                 nig_values_list_imp[d] = {}
#             nig_values_list_imp[d][nit] = l
# nig_values_list_imp

# data -> list of (non_inc_tabu/non_inc_globals, improvement)
nig_nit_ratio_imp: dict[str, list[tuple[float, float]]] = {}
for d in nig_values_map:
    l = []
    for nit in nig_values_map[d]:
        nigs = sorted(nig_values_map[d][nit].keys())
        best_v = nig_values_map[d][nit][nigs[0]]
        for nig in nigs:
            v = nig_values_map[d][nit][nig]
            if v < best_v:
                l.append(((nig/nit), 100*(best_v-v)/best_v))
                best_v = v
    if len(l) != 0:
        nig_nit_ratio_imp[d] = l

# n -> (nig/nit, imp)
points_w_map: dict[tuple[int, float], float] = {}
for d in nig_nit_ratio_imp:
    for pair in nig_nit_ratio_imp[d]:
        if (size_of(d), pair[0]) in points_w_map:
            points_w_map[(size_of(d), pair[0])].append(pair[1])
        else:
            points_w_map[(size_of(d), pair[0])] = [pair[1]]
points_w_map = {k: avg(points_w_map[k]) for k in points_w_map}
points_w: list[tuple[int, float, float]] = [
    (k[0], k[1], points_w_map[k]) for k in points_w_map]
points_w.sort()

with open('report/data/nig_imp.dat', 'w') as f:
    for p in points_w:
        f.write("\t".join([str(n) for n in p])+"\n")


In [17]:
lb_of: dict[str, float] = {}
opt: dict[int, list[(float, float, float)]] = {}
for (d, n, lb, ub, t) in raw_exact:
    if n not in opt:
        opt[n] = []
    opt[n].append((t, ub, lb))
    lb_of[d] = lb

keys = sorted(opt)
for n in keys:
    with open(f'report/data/opt_{n}.dat', 'w') as f:
        for p in opt[n]:
            f.write(f"{p[0]}\t{p[1]/p[2]}\n")
        f.write("\n")

heu: dict[int, list[(float, float)]] = {}
for (d, n, tb, nit, nig, mi, v, _, t) in raw_tabu:
    if tb != n or nig != 5*nit:
        continue
    if n not in keys:
        continue
    if n not in heu:
        heu[n] = []
    heu[n].append((t, v/lb_of[d]))

for n in keys:
    with open(f'report/data/heu_{n}.dat', 'w') as f:
        for p in sorted(heu[n], reverse=True):
            f.write(f"{p[0]}\t{p[1]}\n")
        f.write("\n")
